In [1]:
import os
from dotenv import load_dotenv
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.openai import OpenAI

c:\VSCode\stb\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('OPENAI_API_BASE')
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

In [ ]:
llm = AzureOpenAI(
    engine="stb-gpt-35-turbo", model="gpt-35-turbo", temperature=0.0
)
response = llm.complete("The sky is a beautiful blue and")
print(response)

## Create Translation Module

In [24]:
translation_prompt = """
You are a professional translator. Determine whether the text is in English or not.

Instructions:
1. If the text is not in English, translate it to English.
2. If the text is in English, output the original text without translation or changes.
3. Refrain from adding any additional comments or information to the text.
"""

In [25]:
chat_engine = SimpleChatEngine.from_defaults(
    llm=llm,
    system_prompt=translation_prompt
)

In [27]:
incoming_email_chinese = """
在中国境内飞猪平台上购买门票后发现信息有误后无法修改 现门票二维码有被盗用售卖风险 无法联系上商家进行后续处理 望新加坡文旅局帮忙解决问题 要求立即联系商家立即废票并退还门票费用
"""

In [26]:
eng_ch_response = chat_engine.chat(incoming_email_chinese)
print(eng_ch_response)

After purchasing tickets on the Fliggy platform within China, I discovered that the information was incorrect and cannot be modified. Now there is a risk of the ticket QR code being stolen and resold, and I am unable to contact the merchant for further assistance. I hope that the Singapore Tourism Board can help resolve this issue. I request immediate contact with the merchant to cancel the tickets and refund the ticket fees.


In [9]:
incoming_email_japanese = """
マーライオンをモチーフとしたオリジナルキャラクターの使用許可を頂きたいです。

はじめまして。
イラストレーターの長坂康平(活動名ナガサカ)と申します。

toastyandtipsyという、オーダーメイドで結婚式のペーパーアイテムを制作しているショップ様から、マーライオンをモチーフとしたキャラクターデザイン、及びそのキャラクターを使用したペーパーアイテムの制作の業務委託を受けました。

シンガポールのことが好きな新郎新婦様から、是非マーライオンのキャラクターで結婚式を行いたいという強いご要望がありました。

新郎新婦様へのペーパーアイテムの販売、及び結婚式の使用のみの使用になります。

以上の条件の元、マーライオンのデザイン使用を許諾いただけないでしょうか。よろしくお願
"""

In [10]:
eng_jap_response = chat_engine.chat(incoming_email_japanese)
print(eng_jap_response)

Hello,
I would like to request permission to use a character design based on the Merlion.

My name is Yasuhei Nagasaka, also known as illustrator Nagasaka. I have been commissioned by a shop called toastyandtipsy, which specializes in creating custom wedding paper items, to create a character design based on the Merlion and to produce paper items featuring this character.

The bride and groom, who have a strong affinity for Singapore, expressed a strong desire to have a wedding featuring the Merlion character.

The use of the paper items will be limited to selling them to the bride and groom and using them for their wedding.

Under these conditions, would it be possible to grant permission for the use of the Merlion design? Thank you very much.


In [11]:
incoming_email_korean = """
안녕하십니까

저희는 한국산림복지진흥원이라는 기관입니다.

한국산림복지진흥원은 숲과 함께 국민행복을 키우는 산림복지전문기관으로 도약하여 온 국민이 산림복지서비스를 향유할 수 있는 시대를 만들기 위해 노력하고 있습니다.

2023년에 열린 웰니스 페스티벌에 참여하지 못해 아쉬움이 컸습니다.

혹시 2024년에도 제 3회 웰니스 페스티벌이 개최예정인지 문의드립니다.

감사합니다.
"""

In [12]:
eng_kor_response = chat_engine.chat(incoming_email_korean)
print(eng_kor_response)

Hello,

We are the Korea Forest Welfare Promotion Institute.

The Korea Forest Welfare Promotion Institute is a specialized organization dedicated to promoting forest welfare and enhancing the happiness of the people through forests. We are striving to create an era where all citizens can enjoy forest welfare services.

We were disappointed that we couldn't participate in the Wellness Festival held in 2023. We would like to inquire if the 3rd Wellness Festival is scheduled to be held in 2024.

Thank you.


## Email Decomposition

In [13]:
from llama_index.core import PromptTemplate

In [14]:
type(eng_jap_response.response)

str

In [15]:
print(eng_jap_response.response)

Hello,
I would like to request permission to use a character design based on the Merlion.

My name is Yasuhei Nagasaka, also known as illustrator Nagasaka. I have been commissioned by a shop called toastyandtipsy, which specializes in creating custom wedding paper items, to create a character design based on the Merlion and to produce paper items featuring this character.

The bride and groom, who have a strong affinity for Singapore, expressed a strong desire to have a wedding featuring the Merlion character.

The use of the paper items will be limited to selling them to the bride and groom and using them for their wedding.

Under these conditions, would it be possible to grant permission for the use of the Merlion design? Thank you very much.


In [16]:
query_gen_prompt_str = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

In [17]:
def generate_queries(llm, query_str: str, num_queries: int = 4):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

queries = generate_queries(llm, eng_jap_response.response, num_queries=4)

print(queries)

['1. What is the significance of the Merlion in Singapore?', '2. How can I obtain permission to use a character design for commercial purposes?', '3. Are there any copyright restrictions on using the Merlion design?', '4. What are the legal requirements for selling custom wedding paper items featuring a copyrighted character design?']


In [18]:
type(queries)

list

## Build Vector Store

In [19]:
# Using a simple vector store
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

In [20]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

In [21]:
embed_model

AzureOpenAIEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000024413202150>, num_workers=None, additional_kwargs={}, api_key='bc436823b2b2405ab95361669b304901', api_base='https://oai-ladp.openai.azure.com/', api_version='2023-07-01-preview', max_retries=10, timeout=60.0, default_headers=None, reuse_client=True, dimensions=None, azure_endpoint='https://oai-ladp.openai.azure.com/', azure_deployment=None, azure_ad_token_provider=None)

In [22]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
documents

[Document(id_='c1a62b24-cdb3-4415-bca8-cb39d813c3fa', embedding=None, metadata={'file_path': 'c:\\VSCode\\stb\\data\\SOP_Links.txt', 'file_name': 'SOP_Links.txt', 'file_type': 'text/plain', 'file_size': 8304, 'creation_date': '2024-05-23', 'last_modified_date': '2024-05-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Email Enquiries\r\n1. Email social media collaborations\r\nhttps://orchid.stb.gov.sg/display/INDUSTRY/General+SOP+-+CC (expand social media collaborations)\r\nGuidelines\r\nStep 1: Filtering\r\n•\tProfiles with 300k followers and up to re-route to STB teams (HQ/RO)\r\n•\tAnything below may proceed with template response (Please refer below)\r\n•\tRequests that come in from legitimate Brands and company profiles may be prioritised and s

In [ ]:
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [ ]:
# Check that the retriever is working

query = "What is the tour offered by Singapore Airlines?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print("Query was:", query)
print("Answer was:", answer)

Query was: What is the tour offered by Singapore Airlines?
Answer was: I'm sorry, but the given context information does not provide any information about the tour offered by Singapore Airlines.


In [ ]:
# Show evidence - currently default k is 2 so 2 sources are shown
print(answer.get_formatted_sources())

> Source (Doc id: c8e5d426-5b8e-4635-a5fd-ccc347c9a52e): Email Enquiries
1. Email social media collaborations
https://orchid.stb.gov.sg/display/INDUSTRY...

> Source (Doc id: a74e9fe9-5704-45a7-a185-6f49d559dc44): Step 3b. If the case is assessed as a breach of TA Act. Forward MOP's correspondence and informat...


In [ ]:
# save index to disk - already done
# index.set_index_id("vector_index")
# index.storage_context.persist("./storage")

In [ ]:
# load index from disk
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="storage")
# load index
index = load_index_from_storage(storage_context, index_id="vector_index")

## Run Queries Over Vector Store

In [34]:
from tqdm.asyncio import tqdm

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

In [35]:
# Create a retriever, increased k to 5 from the default of 2
vector_retriever = index.as_retriever(similarity_top_k=2)

NameError: name 'index' is not defined

In [ ]:
results_dict = await run_queries(queries, [vector_retriever])

100%|██████████| 4/4 [00:01<00:00,  3.16it/s]


In [ ]:
results_dict

{('1. What is the significance of the Merlion in Singapore?',
  0): [NodeWithScore(node=TextNode(id_='cd70823c-fb09-4819-b605-1fb511cae0dc', embedding=None, metadata={'file_name': 'Attrations enquiries.docx', 'file_path': 'c:\\Users\\Admin\\Desktop\\Admin\\Work\\Coding\\ladp\\stb\\data\\Attrations enquiries.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 20068, 'creation_date': '2024-05-19', 'last_modified_date': '2024-05-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='48fa7c67-05d9-4120-ad7c-5d773acd443e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'Attrations enquiries.docx', 'file_path': 'c:\\Users\\Admin\\Desktop\\Admin\

In [ ]:
# Function to extract text from nodes
def extract_text_from_results(results_dict):
    extracted_text = {}
    for key, nodes in results_dict.items():
        extracted_text[key] = [node.node.text for node in nodes]
    return extracted_text

# Extract the text
extracted_text = extract_text_from_results(results_dict)

In [ ]:
# Print the extracted text for the first key
for text in extracted_text[('1. "Merlion character design for wedding paper items"', 0)]:
    print(text)

KeyError: ('1. "Merlion character design for wedding paper items"', 0)

In [ ]:
# Print the extracted text for the first key
for text in extracted_text[('2. "Permission to use Merlion design for wedding paper items"', 1)]:
    print(text)

Japanese Enquiry 1 – In this case, RO was unable to answer MoP’s enquiry, hence, CSO replied MoP in English

マーライオンをモチーフとしたオリジナルキャラクターの使用許可を頂きたいです。

はじめまして。
イラストレーターの長坂康平(活動名ナガサカ)と申します。

toastyandtipsyという、オーダーメイドで結婚式のペーパーアイテムを制作しているショップ様から、マーライオンをモチーフとしたキャラクターデザイン、及びそのキャラクターを使用したペーパーアイテムの制作の業務委託を受けました。

シンガポールのことが好きな新郎新婦様から、是非マーライオンのキャラクターで結婚式を行いたいという強いご要望がありました。

新郎新婦様へのペーパーアイテムの販売、及び結婚式の使用のみの使用になります。

以上の条件の元、マーライオンのデザイン使用を許諾いただけないでしょうか。よろしくお願いいたします。

Translated

nice to meet you.
My name is Kohei Nagasaka (active name: Nagasaka) and I am an illustrator.

We were commissioned by toastyandtipsy, a shop that produces custom-made paper items for weddings, to design a character with a Merlion motif and produce paper items using that character.

We received a strong request from a bride and groom who love Singapore to have their wedding ceremony featuring the Merlion character.

Paper items will be sold to brides and grooms and will only be used at weddings.

Could you please give me perm

In [ ]:
# Warning, this function will concatenate all the text from the results. This may be a lot of text!

def concatenate_text_from_results(results_dict):
    concatenated_text = ""
    for key, nodes in results_dict.items():
        for node in nodes:
            concatenated_text += node.node.text + "\n\n"  # Adding some spacing between entries
    return concatenated_text

# Extract the concatenated text
concatenated_text = concatenate_text_from_results(results_dict)

# Print the concatenated text
print(concatenated_text)

Japanese Enquiry 1 – In this case, RO was unable to answer MoP’s enquiry, hence, CSO replied MoP in English

マーライオンをモチーフとしたオリジナルキャラクターの使用許可を頂きたいです。

はじめまして。
イラストレーターの長坂康平(活動名ナガサカ)と申します。

toastyandtipsyという、オーダーメイドで結婚式のペーパーアイテムを制作しているショップ様から、マーライオンをモチーフとしたキャラクターデザイン、及びそのキャラクターを使用したペーパーアイテムの制作の業務委託を受けました。

シンガポールのことが好きな新郎新婦様から、是非マーライオンのキャラクターで結婚式を行いたいという強いご要望がありました。

新郎新婦様へのペーパーアイテムの販売、及び結婚式の使用のみの使用になります。

以上の条件の元、マーライオンのデザイン使用を許諾いただけないでしょうか。よろしくお願いいたします。

Translated

nice to meet you.
My name is Kohei Nagasaka (active name: Nagasaka) and I am an illustrator.

We were commissioned by toastyandtipsy, a shop that produces custom-made paper items for weddings, to design a character with a Merlion motif and produce paper items using that character.

We received a strong request from a bride and groom who love Singapore to have their wedding ceremony featuring the Merlion character.

Paper items will be sold to brides and grooms and will only be used at weddings.

Could you please give me perm

## Synthesise Final Response

In [ ]:
from string import Template
reply_prompt_template =Template("""
Based on the email content provided below, and the closest relevant information retrieved from our data,
please provide an appropriate response to the email.

Instructions:
1. The tone of the response should be friendly and professional.
2. The response should be relevant to the email content.
3. The response should be concise and clear.
4. The response should be written in English.

Email: 
$email_content

Closest Relevant Information: 
$closest_relevant_info
""")

In [ ]:
stb_system_prompt = """
You are a customer service officer representing the Singapore Tourism Board.
"""

In [ ]:
# Create new instance of chat engine
reply_chat_engine = SimpleChatEngine.from_defaults(
    llm=llm,
    system_prompt=stb_system_prompt
)

In [ ]:
email_content = eng_jap_response.response

In [ ]:
concatenated_text

'Japanese Enquiry 1 – In this case, RO was unable to answer MoP’s enquiry, hence, CSO replied MoP in English\n\nマーライオンをモチーフとしたオリジナルキャラクターの使用許可を頂きたいです。\n\nはじめまして。\nイラストレーターの長坂康平(活動名ナガサカ)と申します。\n\ntoastyandtipsyという、オーダーメイドで結婚式のペーパーアイテムを制作しているショップ様から、マーライオンをモチーフとしたキャラクターデザイン、及びそのキャラクターを使用したペーパーアイテムの制作の業務委託を受けました。\n\nシンガポールのことが好きな新郎新婦様から、是非マーライオンのキャラクターで結婚式を行いたいという強いご要望がありました。\n\n新郎新婦様へのペーパーアイテムの販売、及び結婚式の使用のみの使用になります。\n\n以上の条件の元、マーライオンのデザイン使用を許諾いただけないでしょうか。よろしくお願いいたします。\n\nTranslated\n\nnice to meet you.\nMy name is Kohei Nagasaka (active name: Nagasaka) and I am an illustrator.\n\nWe were commissioned by toastyandtipsy, a shop that produces custom-made paper items for weddings, to design a character with a Merlion motif and produce paper items using that character.\n\nWe received a strong request from a bride and groom who love Singapore to have their wedding ceremony featuring the Merlion character.\n\nPaper items will be sold to brides and grooms and will only be used at weddings.\n\nCo

In [ ]:
# Check the number of tokens from retrieved source, we realise that it explodes gpt-3.5 4k context length

import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(concatenated_text)

4624

In [ ]:
# Do a naive truncation of the retrieved sources

half_length = len(concatenated_text) // 2
truncated_text = concatenated_text[:half_length]

num_tokens_from_string(truncated_text)

2377

In [ ]:
truncated_text

'Japanese Enquiry 1 – In this case, RO was unable to answer MoP’s enquiry, hence, CSO replied MoP in English\n\nマーライオンをモチーフとしたオリジナルキャラクターの使用許可を頂きたいです。\n\nはじめまして。\nイラストレーターの長坂康平(活動名ナガサカ)と申します。\n\ntoastyandtipsyという、オーダーメイドで結婚式のペーパーアイテムを制作しているショップ様から、マーライオンをモチーフとしたキャラクターデザイン、及びそのキャラクターを使用したペーパーアイテムの制作の業務委託を受けました。\n\nシンガポールのことが好きな新郎新婦様から、是非マーライオンのキャラクターで結婚式を行いたいという強いご要望がありました。\n\n新郎新婦様へのペーパーアイテムの販売、及び結婚式の使用のみの使用になります。\n\n以上の条件の元、マーライオンのデザイン使用を許諾いただけないでしょうか。よろしくお願いいたします。\n\nTranslated\n\nnice to meet you.\nMy name is Kohei Nagasaka (active name: Nagasaka) and I am an illustrator.\n\nWe were commissioned by toastyandtipsy, a shop that produces custom-made paper items for weddings, to design a character with a Merlion motif and produce paper items using that character.\n\nWe received a strong request from a bride and groom who love Singapore to have their wedding ceremony featuring the Merlion character.\n\nPaper items will be sold to brides and grooms and will only be used at weddings.\n\nCo

In [ ]:
closest_relevant_info = truncated_text

In [ ]:
formatted_reply_prompt = reply_prompt_template.substitute(email_content=email_content, closest_relevant_info=closest_relevant_info)

In [ ]:
# This is how the prompt going into the final instance of the LLM will look like
formatted_reply_prompt

'\nBased on the email content provided below, and the closest relevant information retrieved from our data,\nplease provide an appropriate response to the email.\n\nInstructions:\n1. The tone of the response should be friendly and professional.\n2. The response should be relevant to the email content.\n3. The response should be concise and clear.\n4. The response should be written in English.\n\nEmail: \nNice to meet you.\nI am illustrator Yasuhei Nagasaka (working name: Nagasaka).\n\nI have received a commission from a shop called "toastyandtipsy" that specializes in creating custom wedding paper items. They have requested a character design based on the Merlion and the production of paper items featuring this character.\n\nThe bride and groom, who have a strong affinity for Singapore, expressed a strong desire to have a wedding featuring the Merlion character.\n\nThe use of the paper items will be limited to sales to the bride and groom and for use at their wedding.\n\nUnder these co

In [ ]:
num_tokens_from_string(formatted_reply_prompt)

2605

In [ ]:
print(formatted_reply_prompt)


Based on the email content provided below, and the closest relevant information retrieved from our data,
please provide an appropriate response to the email.

Instructions:
1. The tone of the response should be friendly and professional.
2. The response should be relevant to the email content.
3. The response should be concise and clear.
4. The response should be written in English.

Email: 
Nice to meet you.
I am illustrator Yasuhei Nagasaka (working name: Nagasaka).

I have received a commission from a shop called "toastyandtipsy" that specializes in creating custom wedding paper items. They have requested a character design based on the Merlion and the production of paper items featuring this character.

The bride and groom, who have a strong affinity for Singapore, expressed a strong desire to have a wedding featuring the Merlion character.

The use of the paper items will be limited to sales to the bride and groom and for use at their wedding.

Under these conditions, would it be

In [ ]:
final_email = reply_chat_engine.chat(message=formatted_reply_prompt)

In [ ]:
print(final_email.response)

Dear Yasuhei Nagasaka,

Thank you for reaching out to us. My name is [Your Name] and I am a customer service officer representing the Singapore Tourism Board. It's a pleasure to meet you.

I understand that you have received a commission from toastyandtipsy, a shop specializing in custom wedding paper items, to create a character design based on the Merlion and produce paper items featuring this character. We also understand that the bride and groom have a strong affinity for Singapore and would like to have a wedding featuring the Merlion character.

In order to use the Merlion design, I would like to inform you that you need to apply for permission through our corporate site. I will provide you with the link to the Merlion page where you can find more information on how to apply for the use of the Merlion design.

[Insert link to the Merlion page on the corporate site]

Please follow the instructions on the page to submit your application. Once your application is received, it will b